In [1]:
import numpy as np
import pandas as pd
import statistics
import scipy.stats as sp
import time
from datetime import timedelta, date
import warnings
from scipy.stats import norm
warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt


FINAL IMPLEMENTATION

STEPS:

simulate 100k random GBM paths
use the same sample of 100k paths to price different instruments. Our paths start with the spot price of the SPY ETF, and assume a RFR and IV taken from the internet. At the time of calculation, the spot is ~ $606, the RFR is ~ 4.25%, and the IV of SPY is ~12%
- up and in Call and Puts
- up and out Cal and Puts
- down and in Call and Puts
- down and out Call and Puts

Run each price with the barrier at varying levels of a given S0: 
- take the strike price = S0
- We can take S0 as the spot price for the SPY ETF (~606 at the time of calculation)
- scale up the barrier by 5% increments of S0:  i.e. b = S0, b = 1.05*S0,... b = 1.45*S0, b = 1.5*S0 
- increment up to a level of 1.5 * S0

Follow the same procedure for "down barriers" starting at 0.95*S0, down to 0.5*S0

For each barrier level, calculate a vanilla call and put option where k = b using both the average of simulated paths, and an exact Black-Scholes pricing methods.
This serves to give us confidence that our paths and methodology are valid. 

In [2]:
def sim_MC_paths(S0, T, r, sigma, n, M):
    """
    Parameters:
    S0: initial price
    T: time to maturity 
    r: risk free rate
    sigma: volatility
    n: number of time steps
    M: number of paths
    """
    paths = []
    dt = T / n
    
    #simulate M paths
    for _ in range(M):
        s = [S0] #declare list of prices 
        #increment n timesteps
        for _ in range(n):
            z = np.random.normal(0,1)
            px = s[-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z) #get next price based on prev. price in list
            s.append(px)
        paths.append(s)
    return paths

def calc_BS_Option_Price(So, sigma, t, K, r, optType):
    d_plus = (np.log(So/K) + (r + (sigma**2)/2)*t)/(sigma*np.sqrt(t))
    d_minus = d_plus - sigma * np.sqrt(t)
    if optType == "call":
        price = So*norm.cdf(d_plus) - K*np.exp(-r*t)*norm.cdf(d_minus)
    elif optType == "put":
        price = K*np.exp(-r*t)*norm.cdf(-d_minus) - So*norm.cdf(-d_plus)
    return price


In [ ]:
# paths = sim_MC_paths(100,0.5,0.05,.2,252,100000) 

KeyboardInterrupt: 

In [18]:
results = pd.DataFrame(columns=['barrier_Scale','in_call','out_call','in_put','out_put','sim_call','sim_put', 'bs_call','bs_put'])

s0 = 120  # Initial stock price
k = 90  # Strike price
r = 0.04  # Risk-free interest rate
sigma = 0.25  # Volatility of the underlying asset
T = 0.5  # Time to maturity in years
n = 252
M = 100000
paths = sim_MC_paths(s0,T,r,sigma,n,M)

s0Scales = [110]

for n in s0Scales:
    b =  n
    kicall = []
    kiput = []
    kocall = []
    koput = []
    vput = []
    vcall = []
    for s in paths:    
        if b > s0: #up options
            if max(s) > b: #knock in
                kicall.append(max(s[-1] - k, 0))
                kocall.append(0)
                kiput.append(max(k - s[-1], 0))
                koput.append(0)
            else: #knock out
                kicall.append(0)
                kocall.append(max(s[-1] - k,0))
                kiput.append(0)
                koput.append(max(k - s[-1],0))
        else: #down options
            if min(s) < b: #knock in 
                kicall.append(max(s[-1] - k, 0))
                kocall.append(0)
                kiput.append(max(k - s[-1], 0))
                koput.append(0)
            else: #knock out
                kicall.append(0)
                kocall.append(max(s[-1] - k,0))
                kiput.append(0)
                koput.append(max(k - s[-1],0))
        #vanilla options with k = b
        vcall.append(max(s[-1] - b, 0))
        vput.append(max(b - s[-1],0))

    price_kicall = np.exp(-r * T) * np.mean(kicall)
    price_kiput = np.exp(-r * T) * np.mean(kiput)
    price_kocall = np.exp(-r * T) * np.mean(kocall)
    price_koput = np.exp(-r * T) * np.mean(koput)
    price_vcall = np.exp(-r * T) * np.mean(vcall)
    price_vput = np.exp(-r * T) * np.mean(vput)
    price_bs_call = calc_BS_Option_Price(s0, sigma, T, b, r, "call")
    price_bs_put = calc_BS_Option_Price(s0, sigma, T, b, r, "put")
    results.loc[len(results)] = [n,price_kicall,price_kocall,price_kiput,price_koput,price_vcall,price_vput,price_bs_call,price_bs_put]



In [19]:
results

,barrier_Scale,in_call,out_call,in_put,out_put,sim_call,sim_put,bs_call,bs_put
0,110.0,12.253018,19.880091,0.301009,0.0,15.568295,3.340168,15.53334,3.355194


TESTING IMPLEMENTATION 

In [ ]:
def barrier_option_pricing(S0, K, B, T, r, sigma, N, M):
    """
    Parameters:
        S0: Initial stock price
        K: Strike price
        B: Barrier level
        T: Time to maturity 
        r: Risk-free interest rate
        sigma: Volatility
        N: Number of time steps
        M: Number of simulations
    """

    dt = T / N
    knockIn = []
    knockOut = []

    for _ in range(M):
        S = [S0]
        for _ in range(N):
            z = np.random.normal(0, 1)
            S.append(S[-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z))
        if B >= S0: #up and in/out
            if max(S) > B:  # Check if the barrier is hit
                knockOut.append(0) #knock out 
                knockIn.append(max(S[-1] - K, 0)) #knock in
            else:
                knockOut.append(max(S[-1] - K, 0))  #knock out
                knockIn.append(0) #knock in
        else: #down and in/out
            if min(S) < B:
                knockOut.append(0) #knock out 
                knockIn.append(max(S[-1] - K, 0)) #knock in
            else:
                knockOut.append(max(S[-1] - K, 0))  #knock out
                knockIn.append(0) #knock in

    kiprice = np.exp(-r * T) * np.mean(knockIn)
    koprice = np.exp(-r * T) * np.mean(knockOut)
    discKI = [i * np.exp(-r * T) for i in knockIn]
    discKO = [i * np.exp(-r * T) for i in knockOut]
    stdErrKI = sp.sem(discKI)
    stdErrKO = sp.sem(discKO)
    return kiprice,koprice, stdErrKI, stdErrKO



In [83]:
# Example usage
S0 = 100
K = 100
B = 200
T = 1
r = 0.05
sigma = 0.2
N = 252
M = 10000

knockin, knockout, kiErr, koErr = barrier_option_pricing(S0, K, B, T, r, sigma, N, M)
print("Up-and-out call option price:", knockin, knockout, kiErr, koErr)


Up-and-out call option price: 0.047349991349125674 10.265800466944233 0.02135490443980646 0.1426809507964907


In [89]:
call, put = monteCarloSimOptions(50,51,0.5,0.05,0.45,252,10000) #n=252
bscall = calc_BS_Option_Price(50,0.45,0.5,51,0.05,"call")
bsput = calc_BS_Option_Price(50,0.45,0.5,51,0.05,"put")
print("MC call: ", call, "MC put: " ,put)
print("BS call: ", bscall, "BS put: " ,bsput)

MC call:  6.652543410673003 MC put:  6.222731050210485
BS call:  6.4345060239691065 BS put:  6.175311537414064
